In [6]:
#RUN
from pandas import DataFrame
import pandas as pd
import numpy as np

In [7]:
#RUN
from jsonapi_client import Session, Filter
import pycurl

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [23]:
#RUN
# 'studies' bisa diganti dengan target semisal 'sample' ; 'MGYS00001939' merupakan accession
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00000709').resource

In [24]:
#RUN
#memunculkan study_name
std

<ResourceObject: studies: MGYS00000709 (2435648516168) (2435648516168)>

In [25]:
#RUN
#melihat jumlah sample pada study di atas
std.samples_count

1

In [26]:
#RUN
#memunculkan list sample sesuai jumlah
std.samples

[<ResourceObject: samples: SRS369430 (2435649617088) (2435649617088)>]

In [ ]:
#RUN/SKIP
sample = std.samples
#melihat informasi pada salah satu sample yang dipilih
x=sample[0]
x.sample_metadata

In [ ]:
#SKIP
blank = []
for x in sample:
    #print(x.sample_metadata)
    df = pd.DataFrame(x.sample_metadata)
    df['key'].tolist()
    print(df)
    #df = pd.DataFrame(x.sample_metadata)
    #print(df)
    #index = np.where(df['key'].str.contains('location'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('biome'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('elevation'))[0]
    #blank.append(df.iloc[index, :])
    #df_new = pd.concat(blank)
#df_new

In [27]:
#RUN
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00000709').resource
print(std.accession)
#melihat informasi pada salah satu sample yang dipilih
sample = std.samples

MGYS00000709


In [28]:
#RUN
#membuat container kosong di bawah :
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []
for x in sample:
    acc.append(x.accession)
    sample_name.append(x.sample_name)
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
    analyze.append(b.accession)
    meta = x.sample_metadata
    meta = pd.DataFrame.from_dict(meta)
    meta = meta.set_index('key') 
    meta = meta.drop(columns='unit')
    meta = meta.transpose()
    df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

analyses: MGYA00038093
analyses: MGYA00038092


In [29]:
#RUN
#reset index agar dapat di merge
df = df.reset_index()
df = df.drop(columns='index')

In [30]:
#RUN
#script merge
merged = acc.merge(sample_name, left_index=True, right_index=True, how='inner')
merged = merged.merge(analyze, left_index=True, right_index=True, how='inner')
merged = merged.merge(df, left_index=True, right_index=True, how='inner')
merged

,ID_Sample,Sample Name,ID_Analyses,NCBI sample classification,instrument model
0,SRS369430,Bacterial community of winter wheat-cultivated...,MGYA00038092,410658,454 GS FLX Titanium


In [ ]:
#RUN SEKALI AJA
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)

In [22]:
#RUN
#masuk ke folder "Study_Jagung"
%cd Study_Jagung
#export ke .csv
merged.to_csv(std.accession+'.csv')

D:\Seminar\Seminar_Anam\Study_Jagung


In [ ]:
#SKIP
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#memasukkan sample yang dipilih ke bentuk kolom
df = pd.DataFrame(sample[0].sample_metadata)
df

In [ ]:
#SKIP
blank = []

In [ ]:
#SKIP
#membuat index baru dari list 'key' yang dipilih
index = np.where(df['key'].str.contains('location'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('biome'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('elevation'))[0]
blank.append(df.iloc[index, :])

In [ ]:
#SKIP
#'key' yang dilih dijadikan data frame baru
df_new = pd.concat(blank)
df_new

In [ ]:
#SKIP
filename_sample = '' 

In [ ]:
#SKIP
#membuat loop accession di dalam bentuk list
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
            #for z in y.downloads:
                #if 'json.biom' in z.url:
                    #print(z.url)

In [ ]:
#SKIP
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)
#%rmdir(Study+str)
#with open('_dir'+i.accession+'.biom', 'wb') as f:
    #c = pycurl.Curl()
    #c.setopt(c.URL, z.url)
    #c.setopt(c.WRITEDATA, f)
    #c.perform()
#c.close()